# **Ekstraksi Tabulasi Diseminasi Menjadi 1 File Excel Dengan Filter Kewilayahan**

Created by Yoga Cahya Putra (BPS Kabupaten Pesawaran)



## Instalasi Library

In [2]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


## Kode Ekstraksi

In [1]:
import pandas as pd
import openpyxl
import os
import datetime

# Folder Tabulasi Pusat
folder_path = r'tambahkan directory folder tabulasi publikasi disini'

# Id Kab Harus sesuai dengan format di tabulasi
target_id_kab = "[1809] PESAWARAN" #pastikan ganti ini ya sobat

# DataFrame 
dfs = {}

# Looping semua files .xlsx di folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx') and not file_name.startswith('~$'):
        file_path = os.path.join(folder_path, file_name)

        # ekstraksi prefix sheet dari nama files
        sheet_prefix = file_name[34:38]
        
        if 'komoditas' in file_name:
            sheet_name = f'{sheet_prefix}_komoditas_kec'
            #penamaan sheet output
            sheet_output_name = file_name[34:53]  # Karakter ke-34 hingga ke-53 dari files name
        else:
            #ekstraksi sheet selain komoditas ga sama jadi ribet wkwkwk
            if 'tabel_5_01' in file_name:
                sheet_name = '5_1_kec'
            else:
                # Default: menggunakan sheet_prefix jika tidak ada kondisi khusus
                sheet_name = f'{sheet_prefix}_kec'
            # sheet output 
            sheet_output_name = file_name[34:38]  # Karakter ke-34 hingga ke-38 dari file name

        try:
            df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
            filtered_df = df[df['id_kab'] == target_id_kab]
            dfs[sheet_output_name] = filtered_df
        except Exception as e:
            print(f'Gagal memproses file {file_name} dengan sheet {sheet_name}: {e}')

# sort sheet by name
sorted_sheet_names = sorted(dfs.keys())

# buat sheet info untuk daftar isi biar gampang ygy
info_df = pd.DataFrame({
    'Tabulasi': sorted_sheet_names,
    'Hyperlink': [f'=HYPERLINK("#{sheet_name}!A1", "{sheet_name}")' for sheet_name in sorted_sheet_names]
})
# get tanggal dan waktu saat ini
now = datetime.datetime.now()

# timestamp
timestamp = now.strftime("%Y%m%d_%H%M%S")

# output files
output_file = rf'ganti ini dengan lokasi penyimpanan/hasil_ekstraksi_{timestamp}.xlsx' #ganti ini ke lokasi penyimpanan yang kamu inginkan sobat eheheh


# write semua sheet
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # sheet info paling awal
    info_df.to_excel(writer, sheet_name='info', index=False)
    # sheet lainnya
    for sheet_name in sorted_sheet_names:
        dfs[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

# open excel untuk modifikasi
wb = openpyxl.load_workbook(output_file)

# tambahkan hyperlink info ke setiap sheet selain sheet info & penjumlahan
for sheet_name in sorted_sheet_names:
    if sheet_name != 'info':
        ws = wb[sheet_name]
        
        # Menambahkan hyperlink ke sheet 'info'
        ws.append([''] * (ws.max_column)) 
        ws.cell(row=ws.max_row, column=1, value=f'=HYPERLINK("#info!A1", "Go to Info")')  # Hyperlink ke sheet 'info'
        
        # Menjumlahkan kolom G hingga kolom terakhir dari baris 2 hingga 12
        col_start_index = openpyxl.utils.column_index_from_string('F')  
        for col in range(col_start_index, ws.max_column):
            ws.cell(row=ws.max_row, column=col + 1, value=f'=SUM({openpyxl.utils.get_column_letter(col + 1)}2:{openpyxl.utils.get_column_letter(col + 1)}12)')

# save lagi ygy biar update
wb.save(output_file)

print(f'File Excel "{output_file}" berhasil dibuat.') #jadi deh


File Excel "/Users/ptrayoga/Sites/localhost/project_bps/Diseminasi UTP/hasil_ekstraksi_20240819_072424.xlsx" berhasil dibuat.
